## JWST Pipeline Validation Testing Notebook: NIRISS/SOSS Time Series Observation, pt. 2
**Instruments affected**: NIRISS

### Table of Contents
1. [Imports](#Imports_ID)

2. [Introduction](#intro)

3. [The `assign_wcs` step](#assign_wcs)

### <a id=#Imports_ID>Imports</a>

Here are the lirbaries being imported here and why:

- `numpy` for numerical calculations.
- `matplotlib.pyplot` for plots.
- `astropy.io.fits` for importing fits files.
- `jwst.pipeline.calwebb_spec2` for using the CalWebb Spec 2 stage.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os

from astropy.io import fits
from jwst.pipeline import calwebb_spec2
from jwst import datamodels

Version of the pipeline we are testing:

In [2]:
import jwst
print(jwst.__version__)

0.16.1


## <a id='intro'> Introduction </a>

In this notebook we'll be testing that the Stage 2 of the CalWebb Spec pipeline works as intended as a whole. We'll stop after every step to quickly check that the results make sense. The whole set of steps ran by TSOs by this first Stage of the CalWebb Pipeline can be found [<a href="https://jwst-pipeline.readthedocs.io/en/latest/jwst/pipeline/calwebb_spec2.html#calwebb-spec2">here</a>]. In what follows, we'll have one section associated to each step, calibrating the data sequentially and exploring the outputs accordingly.

## <a id='groupscale'> The `assign_wcs` step </a>

We first test and validate the `assign_wcs` step. This step simply associates a WCS object with the whole science exposure. Let's add this to the rateint products:

In [4]:
calwebb_spec2.assign_wcs_step.AssignWcsStep.call('output/data_k2-141_superbiasstep_corrected_1_rampfitstep.fits', output_dir='output',save_results=True)

2020-07-30 00:12:39,299 - CRDS - ERROR -  Error determining best reference for 'pars-assignwcsstep'  =   Unknown reference type 'pars-assignwcsstep'
2020-07-30 00:12:39,301 - stpipe.AssignWcsStep - INFO - AssignWcsStep instance created.
2020-07-30 00:12:39,587 - stpipe.AssignWcsStep - INFO - Step AssignWcsStep running with args ('output/data_k2-141_superbiasstep_corrected_1_rampfitstep.fits',).
2020-07-30 00:12:39,588 - stpipe.AssignWcsStep - INFO - Step AssignWcsStep parameters are: {'pre_hooks': [], 'post_hooks': [], 'output_file': None, 'output_dir': 'output', 'output_ext': '.fits', 'output_use_model': False, 'output_use_index': True, 'save_results': True, 'skip': False, 'suffix': None, 'search_output_file': True, 'input_dir': '', 'slit_y_low': -0.55, 'slit_y_high': 0.55}
2020-07-30 00:13:29,914 - stpipe.AssignWcsStep - INFO - COMPLETED assign_wcs
2020-07-30 00:13:40,186 - stpipe.AssignWcsStep - INFO - Saved model in output/data_k2-141_superbiasstep_corrected_1_assignwcsstep.fits
20

<CubeModel(308, 256, 2048) from data_k2-141_superbiasstep_corrected_1_assignwcsstep.fits>

All right! Let's check that the WCS information is on the headers now:

In [7]:
exposure = datamodels.SpecModel('output/data_k2-141_superbiasstep_corrected_1_assignwcsstep.fits')

In [51]:
column, row, order = 2047, 255, 2
ra, dec, wavelength = exposure.meta.wcs(column, row, order)
print(ra,dec,wavelength)

350.9165458333333 -1.189291666666667 0.0
